Ce notebook cherche à explorer et comprendre les donnéees fournies par le RNE et extraire efficacement les informations utiles.

La [documentation officielle](https://www.inpi.fr/sites/default/files/documentation_technique_API_formalit%C3%A9s_v2.5.pdf) est disponible ici.

In [1]:
import pandas as pd
import numpy as np
import json 
import os
from tqdm import tqdm
import re

A quoi ressemble le document ?

In [2]:
json_path = "../../rne/rne_deflated/stock_000001.json"

'''
with open(json_path,"r") as file :

    text = json.load(file)

text[1]
'''

'\nwith open(json_path,"r") as file :\n\n    text = json.load(file)\n\ntext[1]\n'

Visiblement les données sont agglomérées au niveau du code siren (entreprise), pour chaque entreprise il y a une entrée par siret (établissement).

In [3]:
nb_établissements = len(text)
nb_fichiers =len([ file for file in os.listdir("../../rne/rne_deflated") if file.endswith('json')])

print("Il y a {:,} entrées d'entreprises dans le fichier, soit environ {:,} entreprises au total".format(nb_fichiers,nb_fichiers*nb_établissements))

NameError: name 'text' is not defined

In [8]:
df = pd.DataFrame(pd.read_json(json_path)["formality"])

In [180]:
def parse_formality(formality):
    """
    returns 
    - content to parse 
    - Bool
    """

    try:
        if formality.get("typePersonne", "") == 'P':
            return formality.get("content", {}).get("personnePhysique", ""), formality.get("diffusionCommerciale", "")
        
        elif formality.get("typePersonne", "") == 'M':
            return formality.get("content", {}).get("personneMorale", ""), formality.get("diffusionCommerciale", "")
        
        else:
            raise ValueError("Ni personne physique ni personne morale (ex: indivision)")

    except ValueError:
        return None, None


def parse_adresse(BlocAdresse):
    """
    returns
    - Error if not in France
    - codeInseeCommune, adresse
    """

  
    # check si France
    if BlocAdresse.get("pays", "") != "FRANCE":

        raise ValueError("Country is not FRA (France)")

    adresse_elements = [BlocAdresse.get(key, "") for key in ["numVoie", "typeVoie", "voie", "codePostal", "commune"]]

    # cas avec que des valeurs manquantes
    adresse = "" if adresse_elements == ["", "", "", "", ""] else " ".join(adresse_elements)



    return BlocAdresse.get("codeInseeCommune", ""), adresse


def parse_descriptionEtablissement(BlocDescriptionEtablissement):
    """
    returns
    - Error si établissement fermé
    sinon 
    - siret
    - nomCommercial / enseigne
    """

    # cas où l'entreprise est fermée
    if "dateEffetFermeture" in BlocDescriptionEtablissement and BlocDescriptionEtablissement["dateEffetFermeture"] != "null":
        raise ValueError('Etablissement fermé')
    

    # les personnes physiques ne rentrent pas dans ce cadre
    try :
        # on prend l'information la plus complète entre nom commercial et enseigne
        nomCommercial = BlocDescriptionEtablissement.get("nomCommercial", "")
        enseigne = BlocDescriptionEtablissement.get("enseigne", "")
        nomCommercial = nomCommercial if len(nomCommercial) > len(enseigne) else enseigne

    except Exception as e :
        nomCommercial = ""

    return BlocDescriptionEtablissement.get("siret", ""), nomCommercial

def parse_RubriqueEtablissement(rubriqueEtablissement):
    """
    returns
    - None si établissement fermé ou hors de France
    sinon 
    - codeInseeCommune
    - adresse 
    - 
    """

    # cas où l'établissement est fermé
    try:
        siret, nomCommercial = parse_descriptionEtablissement(rubriqueEtablissement.get("descriptionEtablissement", {}))
        
    except ValueError:
        return None
    


    # cas où l'établissement n'est pas en France 
    try:
        codeInseeCommune, adresse = parse_adresse(rubriqueEtablissement.get("adresse", {}))

    except ValueError:
        return None

    print(rubriqueEtablissement)

    # l'activité principale est la première activté
    codeApe = rubriqueEtablissement.get("activites", [{}])[0].get("codeApe", "")

    print("--")
    print(codeApe)

    return dict({'siret': siret, "nomCommercial": nomCommercial, "codeInseeCommune": codeInseeCommune, "adresse": adresse, "codeApe": codeApe})

def parse_content_personne(content):
    """
    returns :
    - liste
    """
    
    etablissements = [content.get("etablissementPrincipal", {})]

    if "autresEtablissements" in content:
        etablissements += content.get("autresEtablissements", [])

    if "etablissementModifie" in content:
        etablissements += content.get("etablissementModifie", [])

    # on prend les infos par établissement avec parse_RubriqueEtablissement 
    parsed_etablissements = [parse_RubriqueEtablissement(etablissement) for etablissement in etablissements if etablissement is not None]

    return parsed_etablissements 

def main_parser(formality):
    
    content, diffusionCommerciale = parse_formality(formality)


    if content is None:
        return None

    parsed_etablissements = parse_content_personne(content)

    return parsed_etablissements[0]

    parsed_etablissements = [etablissement.update({"diffusionCommerciale": diffusionCommerciale}) for etablissement in parsed_etablissements if etablissement is not None]

    return parsed_etablissements

In [181]:
df.formality[115]

{'siren': '005420120',
 'content': {'formeExerciceActivitePrincipale': 'COMMERCIALE',
  'natureCreation': {'dateCreation': '1954-11-03',
   'societeEtrangere': False,
   'formeJuridique': '5599',
   'microEntreprise': False,
   'etablieEnFrance': True,
   'salarieEnFrance': True,
   'relieeEntrepriseAgricole': False,
   'entrepriseAgricole': False,
   'eirl': False},
  'personneMorale': {'adresseEntreprise': {'adresse': {'pays': 'FRANCE',
     'codePays': 'FRA',
     'codePostal': '62140',
     'commune': 'MARCONNELLE',
     'codeInseeCommune': '62550',
     'typeVoie': 'RTE',
     'voie': 'NATIONALE'}},
   'etablissementPrincipal': {'descriptionEtablissement': {'rolePourEntreprise': '1',
     'siret': '00542012000031',
     'indicateurEtablissementPrincipal': True},
    'adresse': {'pays': 'FRANCE',
     'codePays': 'FRA',
     'codePostal': '62140',
     'commune': 'MARCONNELLE',
     'codeInseeCommune': '62550',
     'typeVoie': 'RTE',
     'voie': 'NATIONALE'}},
   'autresEtablisse

In [182]:
main_parser(df.formality[115])

{'descriptionEtablissement': {'rolePourEntreprise': '1', 'siret': '00542012000031', 'indicateurEtablissementPrincipal': True}, 'adresse': {'pays': 'FRANCE', 'codePays': 'FRA', 'codePostal': '62140', 'commune': 'MARCONNELLE', 'codeInseeCommune': '62550', 'typeVoie': 'RTE', 'voie': 'NATIONALE'}}
--

{'descriptionEtablissement': {'rolePourEntreprise': '4', 'siret': '00542012000015', 'indicateurEtablissementPrincipal': False}, 'adresse': {'pays': 'FRANCE', 'codePays': 'FRA', 'codePostal': '80120', 'commune': 'RUE', 'codeInseeCommune': '80688', 'typeVoie': 'RUE', 'voie': 'DE LA FONTAINE'}, 'activites': [{'categoryCode': '02020302', 'indicateurPrincipal': True, 'indicateurProlongement': False, 'dateDebut': '1989-01-27', 'formeExercice': 'COMMERCIALE', 'categorisationActivite1': '02', 'categorisationActivite2': '02', 'categorisationActivite3': '03', 'categorisationActivite4': '02', 'descriptionDetaillee': 'Activités des sièges sociaux', 'indicateurActiviteeApe': True, 'rolePrincipalPourEntrep

{'siret': '00542012000031',
 'nomCommercial': '',
 'codeInseeCommune': '62550',
 'adresse': ' RTE NATIONALE 62140 MARCONNELLE',
 'codeApe': ''}

In [169]:
#dictio.update({"diffusionCommerciale": True})

dictio

{'siret': '00542012000031',
 'nomCommercial': '',
 'codeInseeCommune': '62550',
 'adresse': ' RTE NATIONALE 62140 MARCONNELLE',
 'codeApe': ''}

In [174]:
for i in range(10000):

    res = main_parser(df.formality[i])


In [141]:
df.formality[115]

{'siren': '005420120',
 'content': {'formeExerciceActivitePrincipale': 'COMMERCIALE',
  'natureCreation': {'dateCreation': '1954-11-03',
   'societeEtrangere': False,
   'formeJuridique': '5599',
   'microEntreprise': False,
   'etablieEnFrance': True,
   'salarieEnFrance': True,
   'relieeEntrepriseAgricole': False,
   'entrepriseAgricole': False,
   'eirl': False},
  'personneMorale': {'adresseEntreprise': {'adresse': {'pays': 'FRANCE',
     'codePays': 'FRA',
     'codePostal': '62140',
     'commune': 'MARCONNELLE',
     'codeInseeCommune': '62550',
     'typeVoie': 'RTE',
     'voie': 'NATIONALE'}},
   'etablissementPrincipal': {'descriptionEtablissement': {'rolePourEntreprise': '1',
     'siret': '00542012000031',
     'indicateurEtablissementPrincipal': True},
    'adresse': {'pays': 'FRANCE',
     'codePays': 'FRA',
     'codePostal': '62140',
     'commune': 'MARCONNELLE',
     'codeInseeCommune': '62550',
     'typeVoie': 'RTE',
     'voie': 'NATIONALE'}},
   'autresEtablisse

In [6]:
%%time 

df = pd.DataFrame(pd.read_json(json_path)["formality"])

tqdm.pandas()


df["formality"] = df['formality'].progress_apply(main_parser)

100%|██████████| 99997/99997 [00:00<00:00, 239838.36it/s]


CPU times: user 3.71 s, sys: 10.4 s, total: 14.1 s
Wall time: 17.9 s


In [7]:
df

,formality
0,None
1,None
2,None
3,None
4,None
...,...
99992,[]
99993,[]
99994,[]
99995,[]


In [26]:
df[df.formality != []]

ValueError: ('Lengths must match to compare', (99997,), (0,))

In [31]:
df.formality.unique

<bound method Series.unique of 0        None
1        None
2        None
3        None
4        None
         ... 
99992      []
99993      []
99994      []
99995      []
99996      []
Name: formality, Length: 99997, dtype: object>

In [ ]:
class Entreprise:

    def 